In [12]:
!wc -l goodreads_books.json.gz

'wc' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
!ls -lh | grep goodreads_books.json.gz

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
import gzip

with gzip.open("goodreads_books.json.gz") as f:
    line = f.readline()

In [10]:
line

b'{"isbn": "0312853122", "text_reviews_count": "1", "series": [], "country_code": "US", "language_code": "", "popular_shelves": [{"count": "3", "name": "to-read"}, {"count": "1", "name": "p"}, {"count": "1", "name": "collection"}, {"count": "1", "name": "w-c-fields"}, {"count": "1", "name": "biography"}], "asin": "", "is_ebook": "false", "average_rating": "4.00", "kindle_asin": "", "similar_books": [], "description": "", "format": "Paperback", "link": "https://www.goodreads.com/book/show/5333265-w-c-fields", "authors": [{"author_id": "604031", "role": ""}], "publisher": "St. Martin\'s Press", "num_pages": "256", "publication_day": "1", "isbn13": "9780312853129", "publication_month": "9", "edition_information": "", "publication_year": "1984", "url": "https://www.goodreads.com/book/show/5333265-w-c-fields", "image_url": "https://images.gr-assets.com/books/1310220028m/5333265.jpg", "book_id": "5333265", "ratings_count": "3", "work_id": "5400751", "title": "W.C. Fields: A Life on Film", "t

In [11]:
import json

data = json.loads(line)
data

{'isbn': '0312853122',
 'text_reviews_count': '1',
 'series': [],
 'country_code': 'US',
 'language_code': '',
 'popular_shelves': [{'count': '3', 'name': 'to-read'},
  {'count': '1', 'name': 'p'},
  {'count': '1', 'name': 'collection'},
  {'count': '1', 'name': 'w-c-fields'},
  {'count': '1', 'name': 'biography'}],
 'asin': '',
 'is_ebook': 'false',
 'average_rating': '4.00',
 'kindle_asin': '',
 'similar_books': [],
 'description': '',
 'format': 'Paperback',
 'link': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'authors': [{'author_id': '604031', 'role': ''}],
 'publisher': "St. Martin's Press",
 'num_pages': '256',
 'publication_day': '1',
 'isbn13': '9780312853129',
 'publication_month': '9',
 'edition_information': '',
 'publication_year': '1984',
 'url': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'image_url': 'https://images.gr-assets.com/books/1310220028m/5333265.jpg',
 'book_id': '5333265',
 'ratings_count': '3',
 'work_id': '5400751',
 'title': '

In [14]:
def parse_fields(line):
    data = json.loads(line)
    return {
        "book_id": data["book_id"], 
        "title": data["title_without_series"], 
        "ratings": data["ratings_count"], 
        "url": data["url"], 
        "cover_image": data["image_url"]
    }

In [15]:
books_titles = []
with gzip.open("goodreads_books.json.gz") as f:
    while True:
        line = f.readline()
        if not line:
            break
        fields = parse_fields(line)
        try:
            ratings = int(fields["ratings"])
        except ValueError:
            continue
        if ratings > 5:
            books_titles.append(fields)

In [17]:
import pandas as pd
# books_titles is a list of dictionaries
titles = pd.DataFrame.from_dict(books_titles)

In [18]:
titles.shape

(1782579, 5)

In [19]:
titles["ratings"] = pd.to_numeric(titles["ratings"])

In [20]:
titles["mod_title"] = titles["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)

In [ ]:
# what is this reg ex doing? [^a-zA-Z0-9 ]
# ^ means not
# a-z means all lowercase letters
# A-Z means all uppercase letters
# 0-9 means all numbers
# " " means space
# so this regex is saying "replace everything that is not a letter, number, or space with nothing"

In [21]:
titles["mod_title"] = titles["mod_title"].str.lower()

In [22]:
titles["mod_title"] = titles["mod_title"].str.replace("\s+", " ", regex=True)

In [ ]:
# what is this reg ex doing? \s+
# \s means space
# + means one or more
# so this regex is saying "replace all spaces with one space"

In [23]:
titles = titles[titles["mod_title"].str.len() > 0]

In [24]:
titles.to_json("books_titles.json")

In [25]:
titles

,book_id,title,ratings,url,cover_image,mod_title
0,1333909,Good Harbor,10,https://www.goodreads.com/book/show/1333909.Go...,https://s.gr-assets.com/assets/nophoto/book/11...,good harbor
1,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
2,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
3,287140,Runic Astrology: Starcraft and Timekeeping in ...,15,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,runic astrology starcraft and timekeeping in t...
4,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
...,...,...,...,...,...,...
1782574,3084038,"This Sceptred Isle, Vol. 10: The Age of Victor...",12,https://www.goodreads.com/book/show/3084038-th...,https://images.gr-assets.com/books/1494763458m...,this sceptred isle vol 10 the age of victoria ...
1782575,26168430,Sherlock Holmes and the July Crisis,6,https://www.goodreads.com/book/show/26168430-s...,https://images.gr-assets.com/books/1440592011m...,sherlock holmes and the july crisis
1782576,2342551,The Children's Classic Poetry Collection,36,https://www.goodreads.com/book/show/2342551.Th...,https://s.gr-assets.com/assets/nophoto/book/11...,the childrens classic poetry collection
1782577,22017381,"101 Nights: Volume One (101 Nights, #1-3)",70,https://www.goodreads.com/book/show/22017381-1...,https://images.gr-assets.com/books/1398621236m...,101 nights volume one 101 nights 13


In [26]:
# inverse document frequency
# tfidf = term frequency * inverse document frequency
# term frequency = number of times a word appears in a document
# inverse document frequency = log(number of documents / number of documents containing the word)
# if a word appears in every document, the inverse document frequency is 0, so the tfidf is 0, because the word is not important
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

tfidf = vectorizer.fit_transform(titles["mod_title"])

In [27]:
# build a search engine
# given a query, find the 5 most similar books
# similarity = cosine similarity
# cosine similarity = dot product of two vectors / product of the magnitudes of the two vectors
# dot product = sum of the products of the corresponding elements of two vectors
# magnitude = square root of the sum of the squares of the elements of a vector
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

def search(query,vectorizer):
    processed = re.sub("[^a-zA-Z0-9 ]", "", query.lower())
    query_vec = vectorizer.transform([processed])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    # what is np.argpartition(similarity, -10)[-10:] doing?
    # np.argpartition(similarity, -10) returns the indices of the 10 largest elements in similarity
    # [-10:] returns the last 10 elements of the array
    indices = np.argpartition(similarity, -10)[-10:]
    results = titles.iloc[indices]
    results = results.sort_values("ratings", ascending=False)
    
    return results.head(5).style.format({'url': make_clickable, 'cover_image': show_image})

In [28]:
search("harry potter and the prisoner of azkaban", vectorizer)

,book_id,title,ratings,url,cover_image,mod_title
754545,464164,"Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)",22794,Goodreads,,harry potter and the prisoner of azkaban harry potter 3
1236294,28765183,Harry Potter and the Prisoner of Azkaban,3066,Goodreads,,harry potter and the prisoner of azkaban
343661,1132457,"Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)",372,Goodreads,,harry potter and the prisoner of azkaban harry potter 3
1202032,1516338,"Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)",246,Goodreads,,harry potter and the prisoner of azkaban harry potter 3
440131,13564690,Harry Potter and the Prisoner of Azkaban,216,Goodreads,,harry potter and the prisoner of azkaban


In [40]:
query = "harry potter and the prisoner of azkaban"
processed = re.sub("[^a-zA-Z0-9 ]", "", query.lower())
processed

'harry potter and the prisoner of azkaban'

In [41]:
query_vec = vectorizer.transform([processed])
similarity = cosine_similarity(query_vec, tfidf).flatten()
similarity

array([0.        , 0.03488748, 0.        , ..., 0.01512856, 0.        ,
       0.00695407])

In [48]:
indices = np.argpartition(similarity, -10)[-10:]
indices

array([1195819,  437161, 1297772,  341913,  750595,  833806,  437878,
       1675969,  524412, 1229923], dtype=int64)

In [49]:
results = titles.iloc[indices]
results.sort_values("ratings", ascending=False)

,book_id,title,ratings,url,cover_image,mod_title
754545,464164,Harry Potter and the Prisoner of Azkaban (Harr...,22794,https://www.goodreads.com/book/show/464164.Har...,https://images.gr-assets.com/books/1310384602m...,harry potter and the prisoner of azkaban harry...
1236294,28765183,Harry Potter and the Prisoner of Azkaban,3066,https://www.goodreads.com/book/show/28765183-h...,https://s.gr-assets.com/assets/nophoto/book/11...,harry potter and the prisoner of azkaban
343661,1132457,Harry Potter and the Prisoner of Azkaban (Harr...,372,https://www.goodreads.com/book/show/1132457.Ha...,https://images.gr-assets.com/books/1467363945m...,harry potter and the prisoner of azkaban harry...
1202032,1516338,Harry Potter and the Prisoner of Azkaban (Harr...,246,https://www.goodreads.com/book/show/1516338.Ha...,https://s.gr-assets.com/assets/nophoto/book/11...,harry potter and the prisoner of azkaban harry...
440131,13564690,Harry Potter and the Prisoner of Azkaban,216,https://www.goodreads.com/book/show/13564690-h...,https://images.gr-assets.com/books/1332983314m...,harry potter and the prisoner of azkaban
1304461,72190,Harry Potter and the Prisoner of Azkaban (Harr...,110,https://www.goodreads.com/book/show/72190.Harr...,https://s.gr-assets.com/assets/nophoto/book/11...,harry potter and the prisoner of azkaban harry...
439404,30955401,Harry Potter and the Prisoner of Azkaban (Harr...,63,https://www.goodreads.com/book/show/30955401-h...,https://images.gr-assets.com/books/1467859929m...,harry potter and the prisoner of azkaban harry...
527153,23767995,Harry Potter and the Prisoner of Azkaban,49,https://www.goodreads.com/book/show/23767995-h...,https://images.gr-assets.com/books/1442436813m...,harry potter and the prisoner of azkaban
838201,13612284,Harry Potter and the Prisoner of Azkaban,33,https://www.goodreads.com/book/show/13612284-h...,https://images.gr-assets.com/books/1335089204m...,harry potter and the prisoner of azkaban
1684622,32944873,Harry Potter and the Prisoner of Azkaban,7,https://www.goodreads.com/book/show/32944873-h...,https://s.gr-assets.com/assets/nophoto/book/11...,harry potter and the prisoner of azkaban
